# Export analysis grid tiles from an `odc-geo` GridSpec

In [1]:
import geopandas as gpd

from odc.geo.geom import BoundingBox
from odc.geo.gridspec import GridSpec
from odc.geo.types import xy_


def id_to_tuple(id_str):
    """
    Converts a tile ID in form 'x143y56' to a ix, iy tuple so it can
    be passed to a GridSpec (e.g. `gs[ix, iy]`)
    """
    ix, iy = id_str.replace("x", "").split("y")
    return int(ix), int(iy)

## Create GridSpec tile grid

In [2]:
# Load Collection 3 summary grid, reproject to Albers
c3_path = "https://data.dea.ga.gov.au/derivative/ga_summary_grid_c3.geojson"
c3_grid = gpd.read_file(c3_path)

# Extract BoundingBox, converting to ints to avoid floating point issues
c3_bbox = BoundingBox(
    *c3_grid.to_crs("EPSG:3577").total_bounds.round().astype(int), crs="EPSG:3577"
)
c3_bbox

BoundingBox(left=-2400000, bottom=-5088000, right=2784000, top=-864000, crs=CRS('EPSG:3577'))

In [3]:
# Create 32 km tiles using gridspec, using origin used by C3 grid to
# preserve positive indices
gs_32km = GridSpec(
    crs="EPSG:3577",
    resolution=10,
    tile_shape=(3200, 3200),
    origin=xy_(-2688000, -5472000),
)
gs_32km

GridSpec(crs=EPSG:3577, tile_shape=Shape2d(x=3200, y=3200), resolution=Resolution(x=10, y=-10))

## Create vector file outputs

In [4]:
# Convert grid to geopandas polygon dataset covering full C3 grid extent
grid_gdf = gpd.GeoDataFrame.from_features(
    gs_32km.geojson(bbox=c3_bbox), crs="EPSG:4326"
)

# Update column names to match C3 grid
grid_gdf[["ix", "iy"]] = grid_gdf.idx.str.split(",", expand=True).astype(int)
grid_gdf["id"] = "x" + grid_gdf.ix.astype(str) + "y" + grid_gdf.iy.astype(str)

# Copy Collection 3 grid, then slightly shrink geometry to make sure we
# have no ambiguity along grid cell edges for the subsequent spatial join
c3_grid_shrunk = c3_grid[["region_code", "utc_offset", "geometry"]].copy()
c3_grid_shrunk["geometry"] = c3_grid_shrunk.geometry.buffer(-0.01)

# Spatially join attributes from Collection 3 grid to new grid, and
# select a subset of desired columns
grid_gdf_joined = grid_gdf.sjoin(c3_grid_shrunk)[
    ["id", "ix", "iy", "region_code", "utc_offset", "geometry"]
].set_index("id")

/tmp/ipykernel_628/1685867338.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  c3_grid_shrunk["geometry"] = c3_grid_shrunk.geometry.buffer(-0.01)


### Join GEODATA Topo 100k attributes

In [10]:
region_path = "https://data.dea.ga.gov.au/derivative/dea_coastlines/supplementary/cstauscd_r.geojson"
region_gdf = gpd.read_file(region_path)
region_gdf = region_gdf[["FEAT_CODE", "geometry"]].set_index("FEAT_CODE")


In [14]:
# Identify mainland tiles, and add to "type" field
grid_gdf_mainland = grid_gdf_joined.clip(region_gdf.loc[["mainland"]])
grid_gdf_joined["type"] = "offshore"
grid_gdf_joined.loc[grid_gdf_mainland.index, "type"] = "mainland"

### Subset and clip to the coast

In [28]:
mask_path = "https://data.dea.ga.gov.au/derivative/dea_coastlines/supplementary/albers_grids/coastal_mask_15km.geojson"
coastal_mask_gdf = gpd.read_file(mask_path)

In [29]:
# Clip to coastal zone using mask
grid_gdf_clipped = grid_gdf_joined.clip(coastal_mask_gdf)

In [30]:
# Subset to cooastal tiles (without clipping)
grid_gdf_coastal = grid_gdf_joined.loc[grid_gdf_clipped.index]

### Export all outputs

In [31]:
grid_gdf_joined.to_file("ga_summary_grid_c3_32km.geojson")
grid_gdf_coastal.to_file("ga_summary_grid_c3_32km_coastal.geojson")
grid_gdf_clipped.to_file("ga_summary_grid_c3_32km_coastal_clipped.geojson")

## Test GridSpec

In [ ]:
# Test the gridspec on a tile ID
gs_32km[id_to_tuple("x135y101")]

In [ ]:
# Test gridspec on coordinates
gs_32km[gs_32km.pt2idx(x=1904005, y=-3664005)]